<a href="https://colab.research.google.com/github/Ricksooon/analise-rfm-cohort-python/blob/main/analise_superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# -*- coding: utf-8 -*-
"""
Projeto Completo: Análise de Vendas, Cohort e RFM da SuperStore
Versão Aprimorada com Gráficos Profissionais
"""

# --- 1. Importação das Bibliotecas ---
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime

# --- 2. Configuração dos Nomes dos Arquivos ---
ARQUIVO_PEDIDOS = 'orders.csv'
ARQUIVO_CLIENTES = 'customers.csv'
ARQUIVO_PRODUTOS = 'product.csv'
ARQUIVO_LOCALIZACAO = 'location.csv'

# --- 3. Carregamento e Verificação dos Dados ---
print("--- INICIANDO PROJETO DE ANÁLISE SUPERSTORE (VERSÃO APRIMORADA) ---")
arquivos_necessarios = [ARQUIVO_PEDIDOS, ARQUIVO_CLIENTES, ARQUIVO_PRODUTOS, ARQUIVO_LOCALIZACAO]
arquivos_faltando = [f for f in arquivos_necessarios if not os.path.exists(f)]

if arquivos_faltando:
    print(f"\nERRO CRÍTICO: Arquivos não encontrados: {', '.join(arquivos_faltando)}")
    print("Por favor, faça o upload de TODOS os quatro arquivos CSV para o ambiente do Colab.")
else:
    print("Todos os arquivos foram encontrados! Carregando e unindo as tabelas...")
    df_pedidos = pd.read_csv(ARQUIVO_PEDIDOS)
    df_clientes = pd.read_csv(ARQUIVO_CLIENTES)
    df_produtos = pd.read_csv(ARQUIVO_PRODUTOS)
    df_localizacao = pd.read_csv(ARQUIVO_LOCALIZACAO)

    # --- 4. União (Merge) das Tabelas ---
    df = pd.merge(df_pedidos, df_clientes, on='Customer ID', how='left')
    df = pd.merge(df, df_produtos, on='Product ID', how='left')
    df = pd.merge(df, df_localizacao, on='Order ID', how='left')

    # --- 5. Preparação dos Dados ---
    df['Order Date'] = pd.to_datetime(df['Order Date'])
    print("Dados preparados. Iniciando as análises...")

    # ==============================================================================
    # ANÁLISE 1: DESEMPENHO DE VENDAS MENSAIS (GRÁFICO APRIMORADO)
    # ==============================================================================
    print("\n--- Gerando Análise 1: Desempenho de Vendas (Gráfico Aprimorado) ---")
    df_vendas = df.copy()
    df_vendas.set_index('Order Date', inplace=True)
    monthly_sales = df_vendas['Sales'].resample('ME').sum()

    plt.style.use('seaborn-v0_8-whitegrid')
    plt.figure(figsize=(16, 8))
    ax1 = sns.lineplot(x=monthly_sales.index, y=monthly_sales.values, marker='o', markersize=8, label='Receita Mensal')

    # APRIMORAMENTO: Adiciona linha de média
    media_mensal = monthly_sales.mean()
    ax1.axhline(media_mensal, color='red', linestyle='--', label=f'Média Mensal: R$ {media_mensal:,.2f}'.replace(',', '.'))

    # APRIMORAMENTO: Anotações de pico e vale
    pico_vendas = monthly_sales.idxmax()
    vale_vendas = monthly_sales.idxmin()
    ax1.annotate(f'Pico de Vendas\n{pico_vendas.strftime("%b %Y")}', xy=(pico_vendas, monthly_sales.max()), xytext=(pico_vendas, monthly_sales.max() + 15000),
                 arrowprops=dict(facecolor='green', shrink=0.05), ha='center')
    ax1.annotate(f'Menor Venda\n{vale_vendas.strftime("%b %Y")}', xy=(vale_vendas, monthly_sales.min()), xytext=(vale_vendas, monthly_sales.min() - 20000),
                 arrowprops=dict(facecolor='orange', shrink=0.05), ha='center')

    # APRIMORAMENTO: Título e formatação
    data_inicio = monthly_sales.index.min().strftime('%b de %Y')
    data_fim = monthly_sales.index.max().strftime('%b de %Y')
    ax1.set_title(f'Evolução da Receita Mensal ({data_inicio} a {data_fim})', fontsize=20, fontweight='bold', pad=20)
    ax1.set_xlabel('Data do Pedido', fontsize=12)
    ax1.set_ylabel('Receita (R$)', fontsize=12)
    from matplotlib.ticker import FuncFormatter
    def currency_formatter(x, pos): return 'R$ {:,.0f}'.format(x).replace(',', '.')
    ax1.yaxis.set_major_formatter(FuncFormatter(currency_formatter))
    plt.legend()
    plt.tight_layout()
    plt.savefig('grafico_receita_mensal_aprimorado.png', dpi=300)
    plt.close()
    print("--> SUCESSO! Gráfico 'grafico_receita_mensal_aprimorado.png' salvo.")

    # ==============================================================================
    # ANÁLISE 2: RETENÇÃO DE CLIENTES (COHORT - GRÁFICO APRIMORADO)
    # ==============================================================================
    print("\n--- Gerando Análise 2: Retenção de Clientes (Cohort - Gráfico Aprimorado) ---")
    def get_month(x): return datetime(x.year, x.month, 1)
    df['OrderMonth'] = df['Order Date'].apply(get_month)
    df['CohortMonth'] = df.groupby('Customer ID')['OrderMonth'].transform('min')
    order_year, order_month = df['OrderMonth'].dt.year, df['OrderMonth'].dt.month
    cohort_year, cohort_month = df['CohortMonth'].dt.year, df['CohortMonth'].dt.month
    df['CohortIndex'] = (order_year - cohort_year) * 12 + (order_month - cohort_month)
    cohort_data = df.groupby(['CohortMonth', 'CohortIndex'])['Customer ID'].apply(pd.Series.nunique).reset_index()
    cohort_count = cohort_data.pivot_table(index='CohortMonth', columns='CohortIndex', values='Customer ID')
    cohort_size = cohort_count.iloc[:, 0]
    retention = cohort_count.divide(cohort_size, axis=0)
    retention.index = retention.index.strftime('%Y-%m')
    plt.figure(figsize=(18, 10))
    # APRIMORAMENTO: Paleta de cores mais suave, contornos e título descritivo
    sns.heatmap(data=retention, annot=True, fmt='.0%', cmap='Blues', vmin=0.0, vmax=0.5,
                linewidths=.5, linecolor='white')
    plt.title('Taxa de Retenção Mensal de Clientes (Cohort Analysis)', fontsize=20, fontweight='bold', pad=20)
    plt.xlabel('Meses Desde a Primeira Compra', fontsize=12)
    plt.ylabel('Mês da Primeira Compra (Cohort)', fontsize=12)
    plt.tight_layout()
    plt.savefig('grafico_cohort_retencao_aprimorado.png', dpi=300)
    plt.close()
    print("--> SUCESSO! Gráfico 'grafico_cohort_retencao_aprimorado.png' salvo.")

    # ==============================================================================
    # ANÁLISE 3: SEGMENTAÇÃO DE CLIENTES (RFM - GRÁFICO APRIMORADO)
    # ==============================================================================
    print("\n--- Gerando Análise 3: Segmentação de Clientes (RFM - Gráfico Aprimorado) ---")
    snapshot_date = df['Order Date'].max() + pd.Timedelta(days=1)
    rfm_data = df.groupby('Customer ID').agg({
        'Order Date': lambda x: (snapshot_date - x.max()).days,
        'Order ID': 'nunique',
        'Sales': 'sum'
    })
    rfm_data.rename(columns={'Order Date': 'Recency', 'Order ID': 'Frequency', 'Sales': 'MonetaryValue'}, inplace=True)
    quartis = rfm_data.quantile(q=[0.25, 0.5, 0.75])
    def r_score(x, r): return 4 if x <= r['Recency'][0.25] else 3 if x <= r['Recency'][0.50] else 2 if x <= r['Recency'][0.75] else 1
    def fm_score(x, c, fm): return 1 if x <= fm[c][0.25] else 2 if x <= fm[c][0.50] else 3 if x <= fm[c][0.75] else 4
    rfm_data['R_Score'] = rfm_data['Recency'].apply(r_score, args=(quartis,))
    rfm_data['F_Score'] = rfm_data['Frequency'].apply(fm_score, args=('Frequency', quartis,))
    rfm_data['M_Score'] = rfm_data['MonetaryValue'].apply(fm_score, args=('MonetaryValue', quartis,))
    def segment_customer(row):
        if row['R_Score'] >= 4 and row['F_Score'] >= 4: return 'Campeões'
        elif row['R_Score'] >= 3 and row['F_Score'] >= 3: return 'Clientes Fiéis'
        elif row['R_Score'] >= 3 and row['F_Score'] < 3: return 'Potenciais Fiéis'
        elif row['R_Score'] < 3 and row['F_Score'] >= 3: return 'Clientes em Risco'
        elif row['R_Score'] <= 2 and row['F_Score'] <= 2: return 'Hibernando'
        else: return 'Precisam de Atenção'
    rfm_data['Segment'] = rfm_data.apply(segment_customer, axis=1)
    segment_counts = rfm_data['Segment'].value_counts().sort_values(ascending=True)
    total_clientes = len(rfm_data)

    plt.figure(figsize=(14, 8))
    # APRIMORAMENTO: Usando uma paleta de cores gradiente
    colors = sns.color_palette('viridis_r', n_colors=len(segment_counts))
    ax3 = segment_counts.plot(kind='barh', color=colors)
    # APRIMORAMENTO: Adiciona % do total em cada barra
    for index, value in enumerate(segment_counts):
        percentual = (value / total_clientes) * 100
        ax3.text(value, index, f' {value} ({percentual:.1f}%)', va='center', fontweight='bold')

    # APRIMORAMENTO: Título com subtítulo explicativo
    plt.title('Distribuição de Clientes por Segmento RFM', fontsize=20, fontweight='bold', pad=20)
    plt.suptitle('RFM: Recência, Frequência e Valor Monetário', y=0.92, fontsize=12)
    ax3.set_xlabel('Número de Clientes', fontsize=12)
    ax3.set_ylabel('Segmento', fontsize=12)
    plt.tight_layout(rect=[0, 0, 1, 0.9])
    plt.savefig('grafico_rfm_segmentos_aprimorado.png', dpi=300)
    plt.close()
    print("--> SUCESSO! Gráfico 'grafico_rfm_segmentos_aprimorado.png' salvo.")

    print("\n--- PROJETO CONCLUÍDO! Todas as análises aprimoradas foram geradas com sucesso. ---")


--- INICIANDO PROJETO DE ANÁLISE SUPERSTORE (VERSÃO APRIMORADA) ---
Todos os arquivos foram encontrados! Carregando e unindo as tabelas...
Dados preparados. Iniciando as análises...

--- Gerando Análise 1: Desempenho de Vendas (Gráfico Aprimorado) ---
--> SUCESSO! Gráfico 'grafico_receita_mensal_aprimorado.png' salvo.

--- Gerando Análise 2: Retenção de Clientes (Cohort - Gráfico Aprimorado) ---
--> SUCESSO! Gráfico 'grafico_cohort_retencao_aprimorado.png' salvo.

--- Gerando Análise 3: Segmentação de Clientes (RFM - Gráfico Aprimorado) ---
--> SUCESSO! Gráfico 'grafico_rfm_segmentos_aprimorado.png' salvo.

--- PROJETO CONCLUÍDO! Todas as análises aprimoradas foram geradas com sucesso. ---
